---
title: "Shaders with WebGPU"
execute: 
  enabled: true
---

In [1]:
#| code-fold: true
%matplotlib inline

import os

IS_QUARTO = True if os.getenv("IS_QUARTO", "0") == "1" else False
if IS_QUARTO:
    print("Running in Quarto, rendering as Gifs")
    %pip install -qqq matplotlib jupyter-rfb wgpu-shadertoy

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML

from wgpu_shadertoy import Shadertoy

def _mk_gif(shader, end, n, interval, dpi=75):
    def animate(i):
        plt.clf()
        plt.imshow(frames[i])
        plt.axis('off')
        
    frames = [np.array(shader.snapshot(i)) for i in np.linspace(0, end, num=n)]
    resolution = tuple(dim/dpi for dim in frames[0].shape[:2])
    
    fig = plt.figure(figsize=resolution)
    anim = FuncAnimation(
        fig, 
        animate, 
        frames=len(frames),
        interval=interval,  # ms between frames
        repeat=True
    )
    html = anim.to_jshtml(default_mode='loop')
    plt.close()

    # Hide controls
    html = html.replace(
        '<div class="anim-controls">',
        '<div class="anim-controls" style="display: none;">'
    )
    
    # Autoplay gif
    autoplay_script = """
    <script>
        setTimeout(function() {
            var animations = document.getElementsByClassName('animation');
            for (var i = 0; i < animations.length; i++) {
                var buttons = animations[i].getElementsByTagName('button');
                for (var j = 0; j < buttons.length; j++) {
                    if (buttons[j].title === 'Play') {
                        buttons[j].click();
                        break;
                    }
                }
            }
        }, 1000);
    </script>
    """
    
    display(HTML(html + autoplay_script))

def render(cfp, resolution=(250, 250), stacktrace=False, gif=IS_QUARTO, end_time=np.pi, n=100, interval=50):
    code = open(cfp).read() if os.path.exists(cfp) else cfp
    try:
        shader = Shadertoy(code, resolution, offscreen=gif)
        if gif: _mk_gif(shader, end_time, n, interval)
        else: shader.show()
    except Exception as e:
        if stacktrace:
            raise e
        else:
            print("=== ERROR ===\n")
            print(e)

Running in Quarto, rendering as Gifs


/Users/tom/fun/tom-pollak.github.io/.venv/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


## Simple Square

Let's show a color. There seems to be some wonkiness when rendering with Jupyter (I think using Pillow?) it seems to render BGRA rather than RGBA. Tested this same code is rendered as BGRA when running as python using `glfw` renderer.

Anywho we've got a `b2r` function now.

In [2]:
render("""
fn b2r(bgra: vec4<f32>) -> vec4<f32>
{
    return bgra.bgra;
}

fn shader_main(coord: vec2<f32>) -> vec4<f32>
{
    return b2r(vec4<f32>(1.0, 0.0, 0.0, 1.0));
    
}
""")

## RG Color based on pixel position

In [3]:
render("""
fn b2r(bgra: vec4<f32>) -> vec4<f32>
{
    return bgra.bgra;
}

fn shader_main( coord: vec2<f32>  ) -> vec4<f32>
{
    // normalized pixel coordinate (0-1)
    let uv: vec2<f32> = coord / i_resolution.xy;
    // x = red intensity (left to right)
    // y = green intensity (bottom to top)
    let out = vec4<f32>(uv, 0.0, 1.0);
    return b2r(out);
}
""")

## Modulate with time

First animated render! This modulates the intensity using `abs(cos)` (absoltute because -1 intensity is clamped to 0, so we'd rather it just be positive.

In [4]:
render("""
fn b2r(bgra: vec4<f32>) -> vec4<f32>
{
    return bgra.bgra;
}

fn shader_main( coord: vec2<f32>  ) -> vec4<f32>
{
    // normalized pixel coordinate (0-1)
    let uv: vec2<f32> = coord / i_resolution.xy;
    // x = red intensity (left to right)
    // y = green intensity (bottom to top)
    let intensity = abs(cos(i_time)); // 0-1
    let out = vec4<f32>(uv * intensity, 0.0, 1.0);
    return b2r(out);
}
""")

## Square

In [5]:
render("""
fn b2r(bgra: vec4<f32>) -> vec4<f32>
{
    return bgra.bgra;
}

fn square_indicator(coord: vec2<f32>, bound_x: vec2<f32>, bound_y: vec2<f32>) -> f32
{
    if (
        coord.x >= bound_x.x && coord.x <= bound_x.y    // inside x
        && coord.y >= bound_y.x && coord.y <= bound_y.y // inside y
    ) {
        return 1.0;
    }
    return 0.0;
}

fn shader_main(coord: vec2<f32>) -> vec4<f32>
{
    let uv = coord / i_resolution.xy;
    let in_square: f32 = square_indicator(uv, vec2(0.25, 0.75), vec2(0.25, 0.75));
    if (in_square > 0.0) {
        return b2r(vec4(1.0, 0.0, 0.0, 1.0));
    }
    return b2r(vec4(0.0, 0.0, 0.0, 1.0));
    
}
""")

## Simple Circle

In [6]:
render("""
fn b2r(bgra: vec4<f32>) -> vec4<f32>
{
    return bgra.bgra;
}

fn circle_indicator(coord: vec2<f32>, radius: f32) -> f32
{
    // pythagoras
    if ((coord.x * coord.x + coord.y * coord.y) <= (radius * radius)) {
        return 1.0;
    }
    return 0.0;
}

fn shader_main(coord: vec2<f32>) -> vec4<f32>
{
    let uv = coord / i_resolution.xy;
    // our circle is centered around 0, which since we start in the bottom left is 0,0
    // so if we want it centered we need to translate the center to 0,0
    let uv_translate = uv - vec2(0.5, 0.5);
    let in_square: f32 = circle_indicator(uv_translate, 0.25);
    if (in_square > 0.0) {
        return b2r(vec4(1.0, 0.0, 0.0, 1.0));
    }
    return b2r(vec4(0.0, 0.0, 0.0, 1.0));
}
""")

## Better Translate

We had an easy translate function in the previous function, but we're going to want to matrix combine our transformations in a bit so let's make a matmul version.

In [7]:
render("""
fn b2r(bgra: vec4<f32>) -> vec4<f32>
{
    return bgra.bgra;
}

fn mk_translate(x: f32, y: f32) -> mat3x3<f32>
{
    return mat3x3<f32>(
        1.0, 0.0, x,
        0.0, 1.0, y,
        0.0, 0.0, 1.0,
    );
}

fn circle_indicator(coord: vec2<f32>, radius: f32) -> f32
{
    // pythagoras
    if ((coord.x * coord.x + coord.y * coord.y) <= (radius * radius)) {
        return 1.0;
    }
    return 0.0;
}

fn shader_main(coord: vec2<f32>) -> vec4<f32>
{
    let uv = coord / i_resolution.xy;
    // our circle is centered around 0, which since we start in the bottom left is 0,0
    // so if we want it centered we need to translate the center to 0,0
    let uv_translate = uv - vec2(0.5, 0.5);
    let in_square: f32 = circle_indicator(uv_translate, 0.25);
    if (in_square > 0.0) {
        return b2r(vec4(1.0, 0.0, 0.0, 1.0));
    }
    return b2r(vec4(0.0, 0.0, 0.0, 1.0));
}
""")